In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import itertools
from keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
import tools_avg_02 as tools

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()

kernel_size = [5]
num_layers = [10]
batch_size = [30]
learning_rate = [0.0001]
overflow_model = 0

def run(bs, path, lr, ks, num_layer):
    fold=1
    for X_train, Y_train, X_val, Y_val, val_cat in zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label):
        print("Fold "+str(fold))
        model = tools.create_model(lr, bs, ks, num_layer)
        inner_path = path+"/fold_"+str(fold)
        if not os.path.exists(inner_path):
            os.makedirs(inner_path)
        
        early_stop = EarlyStopping(patience=20)
        history = model.fit(x = X_train, 
                            y = Y_train,
                            epochs=80,
                            validation_data=(X_val, Y_val),
                            callbacks=[early_stop],
                            batch_size=bs, 
                            verbose=0)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        validation_prediction = model.predict_classes(X_val, batch_size=bs)
        score = f1_score(val_cat, validation_prediction, average=None)
        
        tools.show_plot(inner_path, history)
        tools.write_file(inner_path+"/readme.txt", evaluation, score, model)
        fold = fold + 1
        del model

## main()
import notify
try:
    base_path = "5_fold_layer_10_avg_02/"
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    index=1
    with open(base_path+"Cartesian_product.txt", "w+") as writer:
        for ks, num_layer, bs, lr in itertools.product(kernel_size, num_layers, batch_size, learning_rate):
            writer.write(str(index)+":"+"\n")
            writer.write('ks={}, num_layer={}, bs={}, lr={}'.format(ks, num_layer, bs, lr)+"\n")
            index = index + 1
        del index

    index=1
    for ks, num_layer, bs, lr in itertools.product(kernel_size, num_layers, batch_size, learning_rate):
        print("Index "+str(index))
        path = base_path+str(index)
        if not os.path.exists(path):
            os.makedirs(path)

        model = tools.create_model(lr, bs, ks, num_layer)
        print(model.summary())
        if model:
            run(bs, path, lr, ks, num_layer)
        else:
            overflow_model = overflow_model + 1
        del model
        index = index + 1

    print("Training finished! Overflow model: ", overflow_model)
except:
    notify.send('Exception occurred: avg_02')
    
notify.send('Finish training! avg_02')

Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)
KFold(n_splits=5, random_state=100, shuffle=True)
trian: [    0     1     2 ... 10148 10149 10150] len 8120 test: [   12    21    22 ... 10136 10137 10138] len 2031
trian: [    2     4     5 ... 10148 10149 10150] len 8121 test: [    0     1     3 ... 10143 10144 10147] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [    4     5     8 ... 10130 10133 10146] len 2030
trian: [    0     1     2 ... 10146 10147 10149] len 8121 test: [   17    23    26 ... 10135 10148 10150] len 2030
trian: [    0     1     3 ... 10147 10148 10150] len 8121 test: [    2     6     7 ... 10142 10145 10149] len 2030
Index 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 8996, 32)          192       
_________________________________________________________________
activation_1 (Activation)    (None, 8996, 32)    

200